# Import and pre-definitions

In [1]:
import os
MAIN_PATH = r'/home/luis-felipe'
DATA_PATH = os.path.join(MAIN_PATH,'data')
PATH_MODELS = os.path.join(MAIN_PATH,'torch_models')
FIGS_PATH = os.path.join(MAIN_PATH,'results','figs')

In [2]:
import torch
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_default_dtype(torch.float64)
SEED = 42
torch.manual_seed(42)
np.random.seed(42)

True


In [4]:
import sys
sys.path.insert(1, '..')
sys.path.insert(1, '../..')

import models
from utils import measures,metrics
from data_utils import upload_logits,split_data
import post_hoc

# Evaluate logits

In [5]:
DATASET = 'ImageNet'
VAL_SIZE = 0.1 #5000
METRIC = metrics.AURC
method = measures.max_logit

In [6]:
p_range = range(8)

d_naurc_p = {p:{} for p in p_range}
d_naurc_msp_p = {p:{} for p in p_range}
d_naurc_opt = defaultdict(list)
d_naurc_msp_opt = defaultdict(list)
d_baseline = defaultdict(list)
acc = defaultdict(list)

In [15]:
naurc_means = {}
naurc_std = {}
baseline_means = {}
for m,v in d_naurc_opt.items():
    naurc_means[m] = np.mean(v)
    baseline_means[m] = np.mean(d_baseline[m])
    naurc_std[m] = np.std(v)

In [7]:
NUM_SPLITS = 10
seed = SEED
for i in range(NUM_SPLITS):
    for model_arc in models.list_models():
        print(model_arc)
        #if model_arc in df['model'].values:
        #    print('ja foi')
        #    continue
        if 'quantized' in model_arc:
            print('quantized - skip')
            continue
        with torch.no_grad():
            try : logits_val,labels_val,logits_test,labels_test = split_data.split_logits(*upload_logits(model_arc,DATASET,PATH_MODELS, 
                                split = 'test', device = dev),VAL_SIZE,seed = seed)
            except: 
                print('cant find')
                continue
        logits_val = post_hoc.centralize(logits_val)
        logits_test = post_hoc.centralize(logits_test)
        risk_val = measures.wrong_class(logits_val,labels_val).float()
        risk_test = measures.wrong_class(logits_test,labels_test).float()
        acc[model_arc].append((1-risk_test.mean().item()))

        for p in p_range:
            d_naurc_p[p][model_arc] = metrics.N_AURC(risk_test,method(post_hoc.normalize(logits_test,p)))
            norm_p = logits_val.norm(p,-1)
            T = post_hoc.optimize.T(logits_val.div(norm_p.reshape(-1,1).div(norm_p.mean())),risk_val).div(norm_p.mean())
            d_naurc_msp_p[p][model_arc] = metrics.N_AURC(risk_test,post_hoc.MSP_p(logits_test,(p,T)))
        p = post_hoc.optimize.p(logits_val,risk_val,metric = METRIC)
        d_naurc_opt[model_arc].append(metrics.N_AURC(risk_test,post_hoc.MaxLogit_p(logits_test,p)))
        pT = post_hoc.optimize.p_and_beta(logits_val,risk_val,metric = METRIC)
        d_naurc_msp_opt[model_arc].append(metrics.N_AURC(risk_test,post_hoc.MSP_p(logits_test,pT)))
        d_baseline[model_arc].append(metrics.N_AURC(risk_test,measures.MSP(logits_test)))
    seed = seed+10
    models_list = list(acc.keys())

alexnet
convnext_base
convnext_large
convnext_small
convnext_tiny
deeplabv3_mobilenet_v3_large
cant find
deeplabv3_resnet101
cant find
deeplabv3_resnet50
cant find
densenet121
densenet161
densenet169
densenet201
efficientnet_b0
efficientnet_b1
efficientnet_b2
efficientnet_b3
efficientnet_b4
efficientnet_b5
efficientnet_b6
efficientnet_b7
efficientnet_v2_l
efficientnet_v2_m
efficientnet_v2_s
fasterrcnn_mobilenet_v3_large_320_fpn
cant find
fasterrcnn_mobilenet_v3_large_fpn
cant find
fasterrcnn_resnet50_fpn
cant find
fasterrcnn_resnet50_fpn_v2
cant find
fcn_resnet101
cant find
fcn_resnet50
cant find
fcos_resnet50_fpn
cant find
googlenet
inception_v3
keypointrcnn_resnet50_fpn
cant find
lraspp_mobilenet_v3_large
cant find
maskrcnn_resnet50_fpn
cant find
maskrcnn_resnet50_fpn_v2
cant find
maxvit_t
mc3_18
cant find
mnasnet0_5
mnasnet0_75
mnasnet1_0
mnasnet1_3
mobilenet_v2
mobilenet_v3_large
mobilenet_v3_small
mvit_v1_b
cant find
mvit_v2_s
cant find
quantized_googlenet
quantized - skip
quantiz

In [8]:
significant = lambda x, epsilon: x*(x > epsilon)
epsilon = 0.01
gains = {}
apg = {}
gains_msp = {}
apg_msp = {}
for p in p_range:
    gains[p] = significant(np.array(list(d_baseline.values()))-np.array(list(d_naurc_p[p].values())),0.01)
    gains_msp[p] = significant(np.array(list(d_baseline.values()))-np.array(list(d_naurc_msp_p[p].values())),0.01)
gains['opt'] = significant(np.array(list(d_baseline.values()))-np.array(list(d_naurc_opt.values())),0.01)
gains_msp['opt'] = significant(np.array(list(d_baseline.values()))-np.array(list(d_naurc_msp_opt.values())),0.01)
for p in gains:
    apg[p] = gains[p].mean()
    apg_msp[p] = gains_msp[p].mean()

ValueError: operands could not be broadcast together with shapes (84,10) (84,) 

In [ ]:
apg

In [ ]:
apg_msp

In [ ]:
baseline = np.array(list(d_baseline.values()))
msp_2 = np.minimum(np.array(list(d_naurc_msp_p[2].values())),baseline-epsilon)
msp_p = np.minimum(np.array(list(d_naurc_msp_opt.values())),baseline-epsilon)
maxlogit_2 = np.array(list(d_naurc_p[2].values()))
maxlogit_p = np.minimum(np.array(list(d_naurc_opt.values())),baseline-epsilon)


In [ ]:
args = np.argsort(maxlogit_p-baseline)#[::-1]
plt.plot((baseline-maxlogit_p)[args], label = 'MaxLogit-p')
#plt.plot((msp_2-maxlogit_2)[args], label = 'MaxLogit-2')
plt.plot((baseline-msp_p)[args], label = 'MSP-p')
plt.plot((baseline-msp_2)[args], label = 'MSP-2')
plt.legend()
plt.grid()
plt.show()

In [ ]:
args = np.argsort(maxlogit_p-msp_2)#[::-1]
plt.plot((msp_2-maxlogit_p)[args], label = 'MaxLogit-pNorm')
#plt.plot((msp_2-maxlogit_2)[args], label = 'MaxLogit-2')
plt.plot((msp_2-msp_p)[args], label = 'MSP-pNorm')
#plt.plot((msp_2-baseline)[args], label = 'Baseline')
plt.legend()
plt.ylim(0,0.01)
plt.grid()
plt.ylabel('Significant gains over MSP-2-Norm')
plt.xlabel('Model')
plt.savefig(os.path.join(FIGS_PATH, f'gains_over_p2.pdf'), transparent = True, format = 'pdf',bbox_inches = 'tight')
plt.show()

In [ ]:
plt.hist(baseline-0.01-np.minimum(maxlogit_p,baseline-0.01),bins = 15)
plt.savefig(os.path.join(FIGS_PATH, f'Histogram_gains.pdf'), transparent = True, format = 'pdf',bbox_inches = 'tight')
plt.show()

In [ ]:
df = pd.concat([pd.DataFrame(apg,index = ['MaxLogit']),pd.DataFrame(apg_msp,index = ['MSP'])])
df

In [ ]:
print(df.transpose().to_latex(float_format="{:.5f}".format))